这个脚本用于在 colab 上使用 llama.cpp 对模型进行量化

In [ ]:
# 安装依赖

# build llama-cpp
!git clone https://github.com/ggerganov/llama.cpp
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config 

In [ ]:
# 配置，使用 colab 参数
source_repo_name = "lenML/aya-expanse-8b-abliterated"
model_name = "aya-expanse-8b-abliterated"

## Download

In [ ]:
from huggingface_hub import snapshot_download

base_model = "./original_model/"
snapshot_download(repo_id=source_repo_name, local_dir=base_model, ignore_patterns=["*.pth"])

## Quantize

In [ ]:
quant_option = "Q4_K_M"
output_model_file = f"/content/models/{model_name}-{quant_option}.gguf"

!cd llama.cpp/build/bin && ./llama-quantize /content/models/model_FP16.gguf {output_model_file} {quant_option}

## Testing

In [ ]:
# install llama-cpp-python
!pip install llama-cpp-python==0.2.85

from llama_cpp import Llama
llm = Llama(model_path=output_model_file)

In [ ]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Q: "
res = llm(prompt, **generation_kwargs)
res

## Uploader

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

target_repo_name = "lenML/aya-expanse-8b-abliterated-GGUF"